#### Alunos: Fernanda Luísa e Igor Michels

In [1]:
using JuMP;
using GLPK;
using Formatting;

# Questão 1

Como o problema fala de valores genéricos para os custos, ($C_1$, $C_2$, ...), colocamos esses valores para ilustrar a formulação do problema.

Para as restrições, podemos ver que $S_1$ e $S_7$ evitam $S_8$, logo, a soma dessas três variáveis deve ser, no máximo, $2$. De modo análogo, $S_3$ ou $S_4$ evita $S_5$, assim, podemos ter $S_3$, $S_4$, $S_3$ e $S_4$ ou $S_5$, logo, $S_3 + S_4 + S_5 \leq 2$ trata todas as possibilidades. Por fim, $\sum_{i = 5} ^ 8 S_i \leq 2$ trata a última restrição.

In [2]:
#       C1, C2, C3, C4, C5, C6, C7, C8, C9, C10
cost = [ 7, 11,  3,  2,  1,  6,  0,  1,  9,  15];
n = size(cost, 1);

A = [1 1 1 1 1 1 1 1 1 1
     1 0 0 0 0 0 1 1 0 0
     0 0 1 1 2 0 0 0 0 0
     0 0 0 0 1 1 1 1 0 0];

b = [5, 2, 2, 2];
s = ['=', "<=", "<=", "<="];

model = Model(GLPK.Optimizer);
@variable(model, x[i = 1:n], base_name = "S", Bin);
cost_function = @expression(model, cost'*x);
@constraint(model, C1, A[1:1, :]*x .== b[1]);
@constraint(model, C2, A[2:4, :]*x .<= b[2:4]);
@objective(model, Min, cost_function);
print(model)

# Questão 2

Tomando o problema como apenas uma universidade, temos o custo dado por $T + \sum_{i = 1}^6 x_it_i$, onde os índices representam os campos de estudo. Podemos linearizar essa função fazendo ela ser da forma $Ty + \sum_{i = 1}^6 x_it_i$, com as restrições $x_i \geq 0$, $y\in \{0, 1\}$ e $\sum_{i = 1}^6 x_i \leq 10y$. Mas isso é para apenas uma universidade.

Como temos 5, podemos fazer o mesmo processo para todas, separando os custos de cada uma das universidades e campos de estudo.

In [3]:
#         C1  C2  C3  C4  C5
costs = [700 800 750 650 700 # custos fixos
         200 225 175 300 250 # custos do primeiro campo de estudo
         100 200 100 250 175 # custos do segundo campo de estudo
         150 125  75 200 150 # custos do terceiro campo de estudo
         175 125 150 200 175 # custos do quarto campo de estudo
          50  25  75 100  75 # custos do quinto campo de estudo
          25  50  50  75  50 # custos do sexto campo de estudo
        ];
costs = transpose(costs);
costs = reshape(costs, (1, 35));
A = [1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0
     0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0
     0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0
     0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0
     0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1
     1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
     1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
     0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
     0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
     0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0
     0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0
     0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
     ];

b = [0, 0, 0, 0, 0, 10, 1, 1, 1, 1, 1, 1];
s = ["<=", "<=", "<=", "<=", "<=", "=", ">=", ">=", ">=", ">=", ">=", ">="];

model = Model(GLPK.Optimizer);
@variable(model, y[i = 1:5], base_name = "y", Bin);
@variable(model, x[i = 1:30] >= 0, base_name = "x", Int);
cost_function = @expression(model, costs[6:35]'*x + costs[1:5]'*y);
@constraint(model, C1, A[1:5, :]*x .<= 10 * y[1:5]);
@constraint(model, C2, A[6:6, :]*x .== b[6:6]);
@constraint(model, C3, A[7:12, :]*x .>= b[7:12]);
@objective(model, Min, cost_function);
print(model)

# Questão 4

## Item a

Podemos modelar o problema de modo similar ao problema de fluxo visto na lista anterior, com as tarefas tendo uma ordem de prioridade. Para isso, definimos $x_{ij}$ como o tempo inicial do processamento do item $i$ na máquina $j$, $x_b$ como o tempo inicial e $x_e$ como o tempo final. Dessa forma, precisamos minimizar $x_e - x_b$ dadas algumas restrições. Seguindo a ideia do problema de fluxo, obtemos as seguintes restrições:
$$x_b \leq x_{11},$$
$$x_b \leq x_{21},$$
$$x_b \leq x_{31},$$
$$x_{11} \leq x_{12}  - t_{11},$$
$$x_{12} \leq x_{13}  - t_{12},$$
$$x_{21} \leq x_{22}  - t_{21},$$
$$x_{22} \leq x_{23}  - t_{22},$$
$$x_{31} \leq x_{32}  - t_{31},$$
$$x_{32} \leq x_{33}  - t_{32},$$
$$x_{13} \leq x_e - t_{13},$$
$$x_{23} \leq x_e - t_{23} \text{ e}$$
$$x_{33} \leq x_e - t_{33}.$$

Essas restrições são suficientes para que evitemos que um item $i$ seja processado pela máquina $j + 1$ antes de ser processado pela máquina $j$. Falta, então, fazer a restrição de que dois itens não estejam na mesma máquina ao mesmo tempo. Para tanto, vamos criar a variável auxiliar, binária, $y_{kij}$ representando que o item $i$ foi processado antes do item $j$ na máquina $k$. Definindo as variáveis $y_{112}$, $y_{113}$ e $y_{123}$ temos todas as possíveis ordenações e, ainda, os dois possíveis ciclos ($1\to 2\to 3\to 1$ e $1\to 3\to 2\to 1$). Para resolver o problema dos ciclos (os quais ocorrem quando $\left(y_{112}, y_{113}, y_{123}\right) \in \{(1, 0, 1), (0, 1, 0)\}$), podemos colocar as restrições adicionais de que
$$y_{112} + y_{123} \geq \dfrac{1}{2}y_{113} \text{ e } 2 - y_{112} - y_{123} \geq \dfrac{1}{2}\left(1 - y_{113}\right).$$

Por fim, temos que, para a máquina $k$, devem ser satisfeitas as seguintes condições
$$x_{1k} \leq x_{2k} - t_{1k} + B\left(x_{1k}, x_{2k}, t_{1k}\right)\cdot \left(1 - y_{k12}\right),$$
$$x_{2k} \leq x_{1k} - t_{2k} + B\left(x_{2k}, x_{1k}, t_{2k}\right)\cdot y_{k12},$$
$$x_{1k} \leq x_{3k} - t_{1k} + B\left(x_{1k}, x_{3k}, t_{1k}\right)\cdot \left(1 - y_{k13}\right),$$
$$x_{3k} \leq x_{1k} - t_{3k} + B\left(x_{3k}, x_{1k}, t_{3k}\right)\cdot y_{k13},$$
$$x_{2k} \leq x_{3k} - t_{2k} + B\left(x_{2k}, x_{3k}, t_{2k}\right)\cdot \left(1 - y_{k23}\right) \text{ e}$$
$$x_{3k} \leq x_{2k} - t_{3k} + B\left(x_{3k}, x_{2k}, t_{3k}\right)\cdot y_{k23},$$

onde $B(\cdot, \cdot, \cdot)$ é uma função dos parâmetros que vai devolver um número (fixo) grande o suficiente para fazer com que as restrições funcionem.

**OBS:** essa função $B$ é, na prática, um número que depende dos parâmetros do problema, de modo que faça com que um cada par de restrições (referentes a um $y$) não seja excludente entre si.

## Item b

Para o item b, onde a ordem dos produtos deve ser a mesma para todas as máquinas, basta fazer $y_{1ij} = y_{2ij} = y_{3ij} = y_{ij}$, o que leva os produtos seguirem, em todas as máquinas, a ordem dada pelas variáveis $_{ij}$.

# Questão 6

### Item a

Nesse exercício, nossa variável de decisão é $x$, a quantidade de commodities, em quilogramas, e com $0 \leq x \leq 50$. A função desejada é linear por partes, pois se $x \leq 20$, temos que maximizar a função $40x - 450$, enquanto para $x \geq 20$ queremos a função $40x - (450 + 5(x - 20)) = 35x - 350$.

Uma solução simples se dá pela divisão do problema em dois, uma para o caso em que $x\leq 20$ e outra para quando $20 \leq x \leq 50$, tomando como solução global o valor de $x$ que dá o maior lucro. Para essas soluções locais, podemos utilizar o simplex.

Agora, resolvendo conforme a ideia descrita no livro (formalização da solução acima), podemos escrever
$$x = \delta_1 + \delta_2,$$
$$20\omega_1 \leq \delta_1 \leq 20,$$
$$0 \leq \delta_2 \leq 30\omega_1 \text{ e}$$
$$\omega_1 \in \{0, 1\},$$

buscando maximizar a função $40\delta_1 + 35\delta_2 - 450$ respeitando as restrições acima. Novamente, podemos utilizar o simplex para resolver.

**OBS:** o problema não especifica a questão de levar meio quilo do commodity, por exemplo, então estamos considerando $\delta_1, \delta_2 \in \mathbb{R}$.

### Item b

Primeiramente, vamos combinar que quando o livro fala em cópias estamos considerando a cópia do artigo completo (as 10 páginas). Assim, o preço é de \\$0.10 para cada uma das primeiras 4 cópias, \\$0.05 para cada uma das 4 próximas cópias e \\$0.025 para as demais cópias do artigo.

Tendo concordado com isso, temos que o problema pode ser definido pela maximização de
$$\dfrac{\delta_1 + \delta_2 + \delta_3}{2} - 0.1\delta_1 - 0.05\delta_2 - 0.025\delta_3 = 0.4\delta_1 + 0.45\delta_2 + 0.475\delta_3,$$

sujeito as condições de
$$4\omega_1 \leq \delta_1 \leq 4,$$
$$4\omega_2 \leq \delta_2 \leq 4\omega_1,$$
$$0 \leq \delta_3 \leq 992\omega_2,$$
$$\omega_2 \leq \omega_1,$$
$$\omega_1 \in \{0, 1\},$$
$$\omega_2 \in \{0, 1\},$$
$$\delta_1 \in \mathbb{Z},$$
$$\delta_2 \in \mathbb{Z} \text{ e}$$
$$\delta_3 \in \mathbb{Z}.$$

Por fim, se considerarmos que o preço de **todas** as cópias muda conforme a quantidade, basta alterar a função que será máximizada para
$$0.4 \delta_1 + 0.45 \delta_2 + 0.475 \delta_3 + 0.2 \omega_1 + 0.2 \omega_2.$$

### Item c

Primeiramente, vamos definir
$$G = \delta_1 + \delta_2 + \delta_3,$$
$$4\omega_1 \leq \delta_1 \leq 4,$$
$$2\omega_2 \leq \delta_2 \leq 2\omega_1,$$
$$0 \leq \delta_3 \leq 2\omega_2,$$
$$\omega_2 \leq \omega_1,$$
$$\omega_1 \in \{0, 1\} \text{ e}$$
$$\omega_2 \in \{0, 1\}.$$

Feito isso, temos que a temperatura utilizada para produzir os galões é dada por
$$2.5\delta_1 + 5\delta_2 + 10\delta_3,$$

logo, como o custo de produção é proporcional a temperatura num fator de $7.5$, o custo total pode ser expresso por
$$7.5\left(2.5\delta_1 + 5\delta_2 + 10\delta_3\right).$$


Por outro lado, o ganho é
$$100\cdot 0.3\left(\delta_1 + \delta_2 + \delta_3\right).$$

Portanto, devemos maximizar
$$100\cdot 0.3\left(\delta_1 + \delta_2 + \delta_3\right) - 7.5\left(2.5\delta_1 + 5\delta_2 + 10\delta_3\right),$$

sujeito as restrições enunciadas acima.

Logo, o total de galões que deve ser produzido é $G = \delta_1 + \delta_2 + \delta_3$.

# Questão 7



Redefine e defina algumas variáveis conforme a imagem abaixo.

<img src = "q7_definitios.jpg">

Adicionalmente, vamos definir a variável $u_{j, i}$, binária, que assume o valor $1$ se o trilho $i$ foi construído na montanha $j$ e $0$ caso contrário. Com isso, vem a restrição de que
$$u_{j, i} \leq u_{j, i - 1}, \forall i \in \{2, \dots, d_j\}.$$

Agora, para cada $\delta_{j, i} \in \left[0, \Delta_{j, i}\right]$, temos o total de pés de trilho que foram colocados no trecho $i$ da montanha $j$. Além disso, podemos ver que o custo desse trecho $i$ da montanha $j$ pode ser expresso, de modo aproximado (bem grosseiramente), por
$$C_{i, j} = C_{i, j_i} \cdot u_{j, i} + \delta_{j, i} \cdot m_{j, i}.$$

Note que $\delta_{j, i}$ pode ser visto como o tamanho do $i$-ésimo trilho da montanha $j$. Dessa forma, temos a restrição de que
$$M = \sum_{k = 1}^j\left(u_{j, 1}\cdot t_{j_0} + \sum_{l = 1}^{d_j} \delta_{k, l}\right).$$

Por fim, vamos adicionar as últimas restrições, onde devemos ter que um novo trecho só será adicionado a montanha se o anterior já foi completo (conforme o gráfico de custo, essa é a interpretação obtida, ou seja, não podemos ter pistas pela metade). Fazendo isso, temos que
$$\forall i \in \{1, \dots, d_j - 1\}, u_{j, 1 + 1}\cdot \Delta_{j, i} \leq \delta_{j, i} \leq u_{j, i}\cdot \Delta_{j, i} \text{ e } 0\leq \delta_{j, d_j}\leq u_{j, d_j}\cdot \Delta_{j, d_j}.$$

Tendo todas as restrições definidas, falta explicitar a função objetivo, que é minimizar o custo. Dessa forma, queremos o mínimo da função
$$\sum_{k = 1}^j \sum_{l = 1}^{d_j} C_{l, k_i} \cdot u_{k, l} + \delta_{k, l} \cdot m_{k, l}.$$

# Músicas - Igor

In [4]:
#           título, autor, dançabilidade, energia, vivacidade e duração (ms)
musics = [["Sereníssima", "Legião Urbana", 0.376, 0.768, 0.355, 241733],
          ["Metal Contra As Nuvens", "Legião Urbana", 0.288, 0.433, 0.112, 689933],
          ["Vinte E Nove", "Legião Urbana", 0.42, 0.542, 0.185, 223933],
          ["Infinita Highway", "Engenheiros Do Hawaii", 0.654, 0.798, 0.136, 372093],
          ["Pra Ser Sincero", "Engenheiros Do Hawaii", 0.482, 0.334, 0.231, 191867],
          ["Era um Garoto, Que Como Eu, Amava os Beatles e os Rolling Stones", "Engenheiros Do Hawaii", 0.568, 0.7, 0.0838, 265040],
          ["Até O Fim - Ao Vivo", "Engenheiros Do Hawaii", 0.635, 0.808, 0.983, 218933],
          ["Natasha - Ao Vivo", "Capital Inicial", 0.374, 0.935, 0.984, 197933],
          ["Primeiros Erros (Chove) - Ao Vivo", "Capital Inicial", 0.246, 0.705, 0.965, 332307],
          ["Lanterna Dos Afogados", "Os Paralamas Do Sucesso", 0.716, 0.596, 0.0966, 189533],
          ["Vital E Sua Moto", "Os Paralamas Do Sucesso", 0.459, 0.878, 0.127, 191216],
          ["O Astronauta de Mármore (Starman)", "Nenhum De Nós", 0.644, 0.637, 0.181, 195533],
          ["Você Vai Lembrar De Mim", "Nenhum De Nós", 0.384, 0.732, 0.173, 289667],
          ["Camila, Camila", "Nenhum De Nós", 0.482, 0.844, 0.335, 312133]];

for music in musics
    println("Nome: ", music[1]);
    println("Artista: ", music[2]);
    println("Dançabilidade: ", music[3]);
    println("Energia: ", music[4]);
    println("Vivacidade: ", music[5]);
    println("Duração (s): ", music[6] / 1000);
    println()
end

Nome: Sereníssima
Artista: Legião Urbana
Dançabilidade: 0.376
Energia: 0.768
Vivacidade: 0.355
Duração (s): 241.733

Nome: Metal Contra As Nuvens
Artista: Legião Urbana
Dançabilidade: 0.288
Energia: 0.433
Vivacidade: 0.112
Duração (s): 689.933

Nome: Vinte E Nove
Artista: Legião Urbana
Dançabilidade: 0.42
Energia: 0.542
Vivacidade: 0.185
Duração (s): 223.933

Nome: Infinita Highway
Artista: Engenheiros Do Hawaii
Dançabilidade: 0.654
Energia: 0.798
Vivacidade: 0.136
Duração (s): 372.093

Nome: Pra Ser Sincero
Artista: Engenheiros Do Hawaii
Dançabilidade: 0.482
Energia: 0.334
Vivacidade: 0.231
Duração (s): 191.867

Nome: Era um Garoto, Que Como Eu, Amava os Beatles e os Rolling Stones
Artista: Engenheiros Do Hawaii
Dançabilidade: 0.568
Energia: 0.7
Vivacidade: 0.0838
Duração (s): 265.04

Nome: Até O Fim - Ao Vivo
Artista: Engenheiros Do Hawaii
Dançabilidade: 0.635
Energia: 0.808
Vivacidade: 0.983
Duração (s): 218.933

Nome: Natasha - Ao Vivo
Artista: Capital Inicial
Dançabilidade: 0.374


Vamos querer um conjunto de músicas de modo que a duração total seja, no máximo, $20$ minutos, sua dançabilidade média seja superior a $0.4$ e de modo que a soma da energia e vivacidade seja máxima.

Para tratar a restrição da dançabilidade, vamos deslocar a média para zero, o que altera a restrição para que a soma das dançabilidades seja superior a $0$.

In [5]:
for i in 1:length(musics)
    musics[i][3] -= 0.4;
end

danceatibility = [musics[i][3] for i in 1:length(musics)];
energy = [musics[i][5] for i in 1:length(musics)];
liveness = [musics[i][5] for i in 1:length(musics)];
duration = [musics[i][6] for i in 1:length(musics)];

Agora, vamos escrever o modelo:

In [6]:
model = Model(GLPK.Optimizer);
@variable(model, selected_musics[1:14], Bin, base_name = "x");
@constraint(model, C1, selected_musics' * danceatibility >= 0);
@constraint(model, C2, selected_musics' * duration <= 1000 * 60 * 20);
@objective(model, Max, selected_musics' * (energy + liveness));
print(model);
optimize!(model);

Por fim, vamos printar as músicas selecionadas:

In [7]:
for i in 1:length(musics)
    if JuMP.value.(selected_musics)[i] == 1
        println("Nome: ", musics[i][1]);
        println("Artista: ", musics[i][2]);
        println("Duração (s): ", musics[i][6] / 1000);
        println();
    end
end

Nome: Sereníssima
Artista: Legião Urbana
Duração (s): 241.733

Nome: Pra Ser Sincero
Artista: Engenheiros Do Hawaii
Duração (s): 191.867

Nome: Até O Fim - Ao Vivo
Artista: Engenheiros Do Hawaii
Duração (s): 218.933

Nome: Natasha - Ao Vivo
Artista: Capital Inicial
Duração (s): 197.933

Nome: Primeiros Erros (Chove) - Ao Vivo
Artista: Capital Inicial
Duração (s): 332.307



# Músicas - Fernanda

In [8]:
#           título, autor, dançabilidade, energia, vivacidade e duração (ms)
musics = [["Too Good At Goodbyes", "Sam Smith", 0.681, 0.372, 0.169, 201000],
          ["Happier", "Ed Sheeran", 0.522, 0.385, 0.135, 207520],
          ["Dive", "Ed Sheeran", 0.761, 0.386, 0.0953, 238440],
          ["In My Blood", "Shawn Mendes", 0.622, 0.712, 0.13, 211360],
          ["Scared to Be Lonely - Acoustic Version", "Martin Garrix", 0.365, 0.2, 0.0804, 257357],
          ["Million Reasons", "Lady Gaga", 0.666, 0.423, 0.106, 205280],
          ["All of Me", "John Legend", 0.422, 0.264, 0.132, 269560],
          ["Easy On Me", "Adele", 0.604, 0.366, 0.133, 224695],
          ["It'll Be Okay", "Shawn Mendes", 0.398, 0.286, 0.0912, 222800],
          ["Visiting Hours", "Ed Sheeran", 0.471, 0.396, 0.0729, 215507],
          ["Naked", "James Arthur", 0.527, 0.61, 0.0572, 233400],
          ["Too Much To Ask", "Niall Horan", 0.443, 0.533, 0.128, 223044],
          ["Say You Won't Let Go", "James Arthur", 0.358, 0.557, 0.0902, 211467],
          ["If I Were a Boy", "Beyoncé", 0.632, 0.518, 0.354, 249147],
          ["No One", "Alicia Keys", 0.644, 0.549, 0.134, 253813],
          ["The Climb", "Miley Cyrus", 0.336, 0.602, 0.141, 234520]]

for music in musics
    println("Nome: ", music[1]);
    println("Artista: ", music[2]);
    println("Dançabilidade: ", music[3]);
    println("Energia: ", music[4]);
    println("Vivacidade: ", music[5]);
    println("Duração (s): ", music[6] / 1000);
    println()
end

Nome: Too Good At Goodbyes
Artista: Sam Smith
Dançabilidade: 0.681
Energia: 0.372
Vivacidade: 0.169
Duração (s): 201.0

Nome: Happier
Artista: Ed Sheeran
Dançabilidade: 0.522
Energia: 0.385
Vivacidade: 0.135
Duração (s): 207.52

Nome: Dive
Artista: Ed Sheeran
Dançabilidade: 0.761
Energia: 0.386
Vivacidade: 0.0953
Duração (s): 238.44

Nome: In My Blood
Artista: Shawn Mendes
Dançabilidade: 0.622
Energia: 0.712
Vivacidade: 0.13
Duração (s): 211.36

Nome: Scared to Be Lonely - Acoustic Version
Artista: Martin Garrix
Dançabilidade: 0.365
Energia: 0.2
Vivacidade: 0.0804
Duração (s): 257.357

Nome: Million Reasons
Artista: Lady Gaga
Dançabilidade: 0.666
Energia: 0.423
Vivacidade: 0.106
Duração (s): 205.28

Nome: All of Me
Artista: John Legend
Dançabilidade: 0.422
Energia: 0.264
Vivacidade: 0.132
Duração (s): 269.56

Nome: Easy On Me
Artista: Adele
Dançabilidade: 0.604
Energia: 0.366
Vivacidade: 0.133
Duração (s): 224.695

Nome: It'll Be Okay
Artista: Shawn Mendes
Dançabilidade: 0.398
Energia:

In [9]:
for i in 1:length(musics)
    musics[i][3] -= 0.5;
end

danceatibility = [musics[i][3] for i in 1:length(musics)];
energy = [musics[i][5] for i in 1:length(musics)];
liveness = [musics[i][5] for i in 1:length(musics)];
duration = [musics[i][6] for i in 1:length(musics)];

In [10]:
model = Model(GLPK.Optimizer);
@variable(model, selected_musics[1:16], Bin, base_name = "x");
@constraint(model, C1, selected_musics' * danceatibility >= 0);
@constraint(model, C2, selected_musics' * duration <= 1000 * 60 * 20);
@objective(model, Max, selected_musics' * (energy + liveness));
print(model);
optimize!(model);

In [11]:
for i in 1:length(musics)
    if JuMP.value.(selected_musics)[i] == 1
        println("Nome: ", musics[i][1]);
        println("Artista: ", musics[i][2]);
        println("Duração (s): ", musics[i][6] / 1000);
        println();
    end
end

Nome: Too Good At Goodbyes
Artista: Sam Smith
Duração (s): 201.0

Nome: Happier
Artista: Ed Sheeran
Duração (s): 207.52

Nome: If I Were a Boy
Artista: Beyoncé
Duração (s): 249.147

Nome: No One
Artista: Alicia Keys
Duração (s): 253.813

Nome: The Climb
Artista: Miley Cyrus
Duração (s): 234.52

